In [1]:
import sys
import os   
import glob

In [2]:
import dotenv
import openai
dotenv.load_dotenv()

True

In [3]:
PARENT_DIR = os.path.dirname(os.getcwd())
src_path = os.path.join(PARENT_DIR, 'src')
sys.path.append(src_path)

DATA_DIR = os.path.join(PARENT_DIR, 'data')

from analyzer import CSVAnalyzer
from graph import Graph

# APIキーを設定
openai.api_key = os.environ.get('OPEN_AI_API_KEY')

In [6]:
from transformers import pipeline
from IPython.display import Audio
import torch
print(torch.__version__)
print(torch.rand(2, 2))

MODEL_NAME = "suno/bark-small"

2.3.0
tensor([[0.1151, 0.5043],
        [0.0362, 0.7322]])


In [10]:
input = "Neurosymbolic Approach: LINC separates the reasoning process into parsing natural language into logical expressions and then deducing the truth value using a symbolic theorem prover. This approach allows for more reliable and accurate logical reasoning compared to methods that rely solely on language models.Performance Evaluation: The effectiveness of LINC is compared across various setups using three different models (Starcoder+, GPT-3.5, and GPT-4) and two datasets (FOLIO and ProofWriter). LINC consistently outperforms traditional LLM setups like Chain-of-Thought prompting, with particularly significant gains noted when using the smaller Starcoder+ model.Error Analysis: The study also includes an analysis of the types of errors encountered using both LINC and traditional methods. It reveals that while both methods exhibit roughly equivalent rates of success, their failure modes are distinct and potentially complementary, suggesting that integrating these approaches could further enhance performance."

In [11]:
def generate_speech(text, model_name="suno/bark-small"):
    pipe = pipeline("text-to-speech", model=model_name)
    output = pipe(text)
    return Audio(output["audio"], rate=output["sampling_rate"])

# Example usage
text = "This is a test using a different voice model."
audio_player = generate_speech(input)
audio_player

/Users/shotomorisaki/Engineering/machine-learning/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/shotomorisaki/Engineering/machine-learning/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/Users/shotomorisaki/Engineering/machine-learning/.venv/lib/python3.12/site-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourc

In [11]:
grade_values = {
    'A': 11, 'A-': 10, 'B+': 9, 'B': 8, 'B-': 7,
    'C+': 6, 'C': 5, 'C-': 4, 'D+': 3, 'D': 2, 'F': 1
}

def calculate_final_grade(test1, redo_test1, test2, redo_test2, test3, redo_test3, test4, test5, hw1, hw2, hw3, hw4):
    # Get test total
    test1 = max(test1, redo_test1)
    test2 = max(test2, redo_test2)
    test3 = max(test3, redo_test3)
    tests = [test1, test2, test3, test4, test5]
    test_total = sum(tests) - min(tests)  # drop the lowest test score

    # Get HW total
    hws = [hw1, hw2, hw3, hw4]
    hw_total = sum(hws) - min(hws)  # drop the lowest homework score

    # Determine minimum passing grade based on test total
    if test_total < 28:
        return 'F'
    elif test_total < 35:
        print('Pass. Minimum grade is C.')
        mingrade = 'C'
    else:  # test_total >= 35
        print('Pass. Minimum grade is B-.')
        mingrade = 'B-'

    # Check your A, B, C grade
    hw_grade = look_up_table(hw_total)

    # Return the higher of the two grades (mingrade or hw_grade)
    final_grade = max(mingrade, hw_grade, key=lambda grade: grade_values[grade])
    return final_grade

def look_up_table(hw_total):
    # Grade lookup based on the provided table
    grade_boundaries = {
        30: 'A', 29: 'A-', 28: 'A-', 27: 'A-', 26: 'A-',
        25: 'A-', 24: 'A-', 23: 'B+', 22: 'B+', 21: 'B+',
        20: 'B', 19: 'B', 18: 'B', 17: 'B-', 16: 'B-',
        15: 'B-', 14: 'C+', 13: 'C+', 12: 'C', 11: 'C',
        10: 'C', 9: 'C', 8: 'C', 7: 'C', 6: 'C',
        5: 'C', 4: 'C', 3: 'C', 2: 'C', 1: 'C', 0: 'C'
    }
    return grade_boundaries.get(hw_total, 'C') 

# Example usage:
print(calculate_final_grade(0, 100, 100, 0, 100, 0, 100, 100, 
                            0, 8, 8, 8))


Pass. Minimum grade is B-.
A-


In [14]:
grade_values = {
    'A': 11, 'A-': 10, 'B+': 9, 'B': 8, 'B-': 7,
    'C+': 6, 'C': 5, 'C-': 4, 'D+': 3, 'D': 2, 'F': 1
}

def calculate_final_grade(test_scores, redo_test_scores, hw_scores):
    # Calculate test and homework totals after adjustments
    test_total = calculate_adjusted_total(test_scores, redo_test_scores)
    hw_total = calculate_adjusted_total(hw_scores)

    # Determine minimum grade based on test total
    mingrade = determine_minimum_grade(test_total)

    # Determine homework grade
    hw_grade = look_up_table(hw_total)

    # Compare and return the higher of the two grades
    final_grade = max(mingrade, hw_grade, key=lambda grade: grade_values[grade])
    return final_grade

def calculate_adjusted_total(scores, redo_scores=None):
    """Calculate the total score after adjustments like dropping the lowest score or considering redo scores."""
    if redo_scores:
        adjusted_scores = [max(score, redo) for score, redo in zip(scores, redo_scores)]
    else:
        adjusted_scores = scores
    return sum(adjusted_scores) - min(adjusted_scores)

def determine_minimum_grade(test_total):
    """Determine the minimum grade based on the test total."""
    if test_total < 28:
        print('Fail: Below minimum test score threshold.')
        return 'F'
    elif test_total < 35:
        print('Pass. Minimum grade is C.')
        return 'C'
    else:
        print('Pass. Minimum grade is B-.')
        return 'B-'

def look_up_table(hw_total):
    """Grade lookup based on homework total."""
    grade_boundaries = {
        30: 'A', 29: 'A-', 28: 'A-', 27: 'A-', 26: 'A-',
        25: 'A-', 24: 'A-', 23: 'B+', 22: 'B+', 21: 'B+',
        20: 'B', 19: 'B', 18: 'B', 17: 'B-', 16: 'B-',
        15: 'B-', 14: 'C+', 13: 'C+', 12: 'C', 11: 'C',
        10: 'C', 9: 'C', 8: 'C', 7: 'C', 6: 'C',
        5: 'C', 4: 'C', 3: 'C', 2: 'C', 1: 'C', 0: 'C'
    }
    return grade_boundaries.get(hw_total, 'C')

# Example usage:
test_scores = [0, 100, 100, 100, 65]
redo_test_scores = [100, 0, 0]
hw_scores = [0, 8, 8, 8]
print(calculate_final_grade(test_scores, redo_test_scores, hw_scores))


Pass. Minimum grade is B-.
A-


In [7]:
from scipy.stats import t
import numpy as np

# Data provided
data = np.array([0.86, 1.53, 1.57, 1.81, 0.99, 1.09, 1.29, 1.78, 1.29, 1.58])

# Calculate mean and standard deviation
mean = np.mean(data)
std_dev = np.std(data, ddof=1)  # using Bessel's correction for sample standard deviation

# Given t-statistic
t_statistic = 1.7273
degrees_of_freedom = 9

# Calculate p-value for the one-tailed test
p_value_one_tailed = 1 - t.cdf(t_statistic, df=degrees_of_freedom)
p_value_two_tailed = 2 * p_value_one_tailed

(mean, std_dev, p_value_one_tailed, p_value_two_tailed)


(1.379, 0.3276668362156225, 0.05909237039922999, 0.11818474079845998)

In [5]:
t.cdf(t_statistic, df=degrees_of_freedom)

0.94090762960077

In [17]:
t.cdf(0.5, 9)

0.6854643500869868

In [3]:
from scipy.stats import t
import math
# Given values
n = 9
sample_mean = 22
hypothesized_mean = 20
sample_std_dev = 3
significance_level = 0.05
sum_squared_deviations = 72

# Calculate the test statistic
test_statistic = (sample_mean - hypothesized_mean) / (sample_std_dev / (n ** 0.5))

# Calculate critical t-value for one-sided test at 0.05 significance level
critical_t_value = t.ppf(1 - significance_level, df=n-1)

# Calculate p-value for the test statistic
p_value = 1 - t.cdf(test_statistic, df=n-1)

(test_statistic, critical_t_value, p_value)


(2.0, 1.8595480375228424, 0.040258118978631297)

In [13]:
# Calculate the sample variance and standard error
s2 = sum_squared_deviations / (n - 1)
s = math.sqrt(s2)
se = s / math.sqrt(n)

# Calculate t-statistics for one-sided and two-sided tests
t_statistic = (sample_mean - hypothesized_mean) / se

# Determine the critical t-values for alpha = 0.05, one-sided and two-sided tests
t_critical_one_sided = -t.ppf(0.05, df=n-1)  # One-sided test (lower tail)
t_critical_two_sided_positive = t.ppf((1-significance_level / 2), df=n-1)  # Two-sided test (upper tail)

s2, se, t_statistic, t_critical_one_sided, t_critical_two_sided_positive


(9.0, 1.0, 2.0, 1.8595480375228428, 2.306004135204166)

In [14]:
# Critical t-value for 95% confidence interval
t_critical = t.ppf(0.975, df=n-1)

# Calculate the confidence interval
lower_bound = sample_mean - t_critical * se
upper_bound = sample_mean + t_critical * se

lower_bound, upper_bound


(19.693995864795834, 24.306004135204166)